In [1]:
import pandas as pd
import datetime as dt
import simplejson as json
from escapejson import escapejson

# SQLAlchemy
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

from sqlalchemy import Column, Integer, String, Float, Date, null, ForeignKey

In [2]:
def fixJson(str):
    companyStr = str.replace("\'", "\"")
    items = companyStr.split(":")
    cleaned = []    
    for item in items:
        commaItems = item.split(",")
        cleanedCommaItems = []
        for commaItem in commaItems:
            left = commaItem.find("\"")
            right = commaItem.rfind("\"")
            #print(f'{commaItem} {left}, {right}')
            while left != right:
                newleft = commaItem.find("\"", left + 1, right - 1)
                right = commaItem.rfind("\"", left + 1, right - 1)
                left = newleft
                #print(f'{commaItem} {left}, {right}')
                if left > 0 and (right == left): #found only one quote
                    #located the isolated quote !!
                    temp = list(commaItem)
                    temp[left] = "'"
                    commaItem = "".join(temp)
                elif left > 0 and right > 0 and (right != left): #found two quotes
                    temp = list(commaItem)
                    temp[left] = "'"
                    temp[right] = "'"
                    commaItem = "".join(temp)
            cleanedCommaItems.append(commaItem)
        cleaned.append(",".join(cleanedCommaItems))
    result = ":".join(cleaned)
    #print(result)
    return result

In [3]:
# Create tables, classes and databases
# ----------------------------------
class Movie(Base):
    __tablename__ = 'movie'
    movie_id = Column(Integer, primary_key=True, autoincrement=False)
    adult = Column(String(10), nullable=True)
    budget = Column(Float, nullable=True)
    imdb_id = Column(String(40), nullable=True)
    language = Column(String(10), nullable=True)
    title = Column(String(512), nullable=True)
    popularity = Column(Float, nullable=True)
    release_date = Column(Date, nullable=True)
    revenue = Column(Float, nullable=True)
    runtime = Column(Float, nullable=True)
    status = Column(String(10), nullable=True)
    vote_average = Column(Float, nullable=True)
    vote_count = Column(Float, nullable=True)

class Rating(Base):
    __tablename__ = 'rating'
    id = Column(Integer, primary_key=True)
    movie_id = Column(Integer, ForeignKey("movie.movie_id")) #foriegn key
    user_id = Column(Integer)
    rating = Column(Float, nullable=True)
    timestamp = Column(Date, nullable=True) 
    
class Genres(Base):
    __tablename__ = 'genres'
    genres_id = Column(Integer, primary_key=True, autoincrement=False)
    geners_name = Column(String(10), nullable=True) 
    
class MovieGenres(Base):
    __tablename__ = 'movie_genres'
    genres_id = Column(Integer, primary_key=True, autoincrement=False)
    movie_id =  Column(Integer, primary_key=True, autoincrement=False) 
    
class ProductionCompanies(Base):
    __tablename__ = 'production_companies'
    production_company_id = Column(Integer, primary_key=True, autoincrement=False)
    production_company_name= Column(String(512), nullable=True) 
    
class MovieProductionCompanies(Base):
    __tablename__ = 'movie_production_companies'
    production_company_id = Column(Integer, primary_key=True, autoincrement=False)
    movie_id =Column(Integer, primary_key=True, autoincrement=False)

In [4]:
# Path to sqlite
database_path = "database/combined_data.sqlite"

# Create an engine that can talk to the database
engine = create_engine(f"sqlite:///{database_path}")
conn = engine.connect()

# Use this to clear out the db
Base.metadata.drop_all(engine)

# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
Base.metadata.create_all(engine)

from sqlalchemy.orm import Session
session = Session(bind=engine)

In [5]:
#import the movies csv file into dataframe
file_movie = "files/movies_metadata.csv"

#columns to import from csv file .. 
col_list = ["adult", "budget", "id", "imdb_id", "original_language", "title", "release_date", "revenue", "runtime", "status", "vote_average", "vote_count", "popularity", "genres", "production_companies" ]


In [6]:
#convert budget to number and remove the non-numeric budget

#remove null or na


#remove zero budget values 


#remove duplicate id from dataframe


#convert id to int and remove the non-numeric ids


#remove null or na



#movies_df.dtypes

In [7]:
#read each row from the dataframe and store in the movie table
print("Starting Movies Migration")


print("Finished Movies Migration")

Starting Movies Migration
Finished Movies Migration


In [8]:
#print(genresDictionary)
#migrate the genres dictionary to the database

#save the session

In [9]:
#print(companiesDictionary)
#migrate the companies dictionary to the database


#save the session

In [10]:
#import the ratings csv file into dataframe

file_ratings = "files/ratings.csv"
ratings_df = pd.read_csv(file_ratings, low_memory=False)
ratings_df.head(5)
ratings_df.dtypes

#read each row from the dataframe and store in the rating table
print("Starting Ratings Migration")


print("Finished Ratings Migration")

Starting Ratings Migration
Finished Ratings Migration


In [11]:
data = pd.read_sql("SELECT * FROM rating", conn)
data.head()

,id,movie_id,user_id,rating,timestamp


In [12]:
data = pd.read_sql("SELECT * FROM movie_genres", conn)
data.head()

,genres_id,movie_id


In [13]:
data = pd.read_sql("SELECT * FROM production_companies", conn)
data.head()

,production_company_id,production_company_name


In [14]:
data = pd.read_sql("SELECT * FROM movie_production_companies", conn)
data.head()

,production_company_id,movie_id


In [15]:
# Saving a New Clean CSV file 


In [16]:
#example query

data = pd.read_sql("SELECT * FROM movie", conn)
data.head()

,movie_id,adult,budget,imdb_id,language,title,popularity,release_date,revenue,runtime,status,vote_average,vote_count


In [17]:
#number of times a movie has been rated
data = pd.read_sql("SELECT * from rating", conn)
data.head()

,id,movie_id,user_id,rating,timestamp


In [18]:
#number of times a movie has been rated
data = pd.read_sql("SELECT movie_id, count(*) as cnt FROM rating group by movie_id", conn)
data.head()

,movie_id,cnt


In [19]:
#average rating for each movie
data = pd.read_sql("SELECT movie_id, avg(rating) as average FROM rating group by movie_id", conn)
data.head()

,movie_id,average


In [20]:
#average rating for each user
data = pd.read_sql("SELECT user_id, avg(rating) as avg FROM rating group by user_id", conn)
data.head()

,user_id,avg


In [21]:
#average rating for each movie
data = pd.read_sql("SELECT movie_id, avg(rating) as average FROM rating group by movie_id order by average desc ", conn)
data.head(100)

,movie_id,average


In [22]:
#histogram of data average
